# Reservoir constraints
To illustrate the usage of the reservoir constraint, we look at an example for scheduling nurses in clinic.

In [1]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

The clinic needs to ensure that there are always enough nurses available without over-staffing too much.
For a 12-hour work day, we model the demands for nurses as integers for each hour of the day. We also have a list of nurses, each with an individual availability as well as a maximum shift length.


In [2]:
demand_change_at_t = [3, 0, 0, 0, 2, 0, 0, 0, -1, 0, -1, 0, -3]
demand_change_times = list(range(len(demand_change_at_t)))

# begin and end of the availability of each nurse
nurse_availabilities = 2 * [
    (0, 7),
    (0, 4),
    (0, 8),
    (2, 9),
    (1, 5),
    (5, 12),
    (7, 12),
    (0, 12),
    (4, 12),
]

max_shift_length = 5

We now initialize all relevant variables of the model. Each nurse is assigned a start and end time of their shift as well as a Boolean variable indicating if they are working at all. We also add some basic constraints to ensure that the shifts are valid.

In [3]:
# boolean variable to indicate if a nurse is scheduled
nurse_scheduled = [
    model.new_bool_var(f"nurse_{i}_scheduled") for i in range(len(nurse_availabilities))
]

# model the begin and end of each shift
shifts_begin = [
    model.new_int_var(begin, end, f"begin_nurse_{i}")
    for i, (begin, end) in enumerate(nurse_availabilities)
]

shifts_end = [
    model.new_int_var(begin, end, f"end_nurse_{i}")
    for i, (begin, end) in enumerate(nurse_availabilities)
]

for begin, end in zip(shifts_begin, shifts_end):
    model.add(end >= begin)  # make sure the end is after the begin
    model.add(end - begin <= max_shift_length)  # make sure, the shifts are not too long

Our reservoir level is the number of nurses scheduled at any time minus the demand for nurses up until that point. We can now add the reservoir constraint to ensure that we have enough nurses available at all times while not having too many nurses scheduled (i.e., the reservoir level is between 0 and 2). We have three types of changes in the reservoir:

1. The demand for nurses changes at the beginning of each hour. For these we use fixed integer times and activate all
   changes. Note that the demand changes are negated, as an increase in demand lowers the reservoir level.
2. If a nurse begins a shift, we increase the reservoir level by 1. We use the `shifts_begin` variables as times and
   change the reservoir level only if the nurse is scheduled.
3. Once a nurse ends a shift, we decrease the reservoir level by 1. We use the `shifts_end` variables as times and
   change the reservoir level only if the nurse is scheduled.

In [4]:
times = demand_change_times
demands = [
    -demand for demand in demand_change_at_t
]  # an increase in demand lowers the reservoir
actives = [1] * len(demand_change_times)

times += list(shifts_begin)
demands += [1] * len(shifts_begin)  # a nurse begins a shift
actives += list(nurse_scheduled)

times += list(shifts_end)
demands += [-1] * len(shifts_end)  # a nurse ends a shift
actives += list(nurse_scheduled)

model.add_reservoir_constraint_with_active(
    times=times,
    level_changes=demands,
    min_level=0,
    max_level=2,
    actives=actives,
)

Finally, we can solve the model and print the results.

In [5]:
solver = cp_model.CpSolver()
status = solver.solve(model)

assert status == cp_model.OPTIMAL

for i in range(len(nurse_availabilities)):
    if solver.Value(nurse_scheduled[i]):
        print(
            f"Nurse {i} is scheduled from {solver.Value(shifts_begin[i])} to {solver.Value(shifts_end[i])}"
        )

Nurse 0 is scheduled from 2 to 7
Nurse 1 is scheduled from 0 to 2
Nurse 2 is scheduled from 3 to 8
Nurse 3 is scheduled from 4 to 9
Nurse 5 is scheduled from 5 to 10
Nurse 6 is scheduled from 11 to 11
Nurse 7 is scheduled from 7 to 12
Nurse 8 is scheduled from 8 to 12
Nurse 9 is scheduled from 0 to 5
Nurse 10 is scheduled from 0 to 4
Nurse 13 is scheduled from 3 to 5
Nurse 14 is scheduled from 5 to 7
Nurse 15 is scheduled from 11 to 11
Nurse 17 is scheduled from 7 to 12
